In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

In [2]:
train_all=pd.read_csv('train_pre.csv')
test=pd.read_csv("test.csv")
user=pd.read_csv("user.csv")
ad=pd.read_csv("ad.csv")
position=pd.read_csv("position.csv")
user_app_actions=pd.read_csv("user_app_actions.csv")
user_installedapps=pd.read_csv("user_installedapps.csv")
app_categories=pd.read_csv("app_categories.csv")
test=pd.merge(test,ad,on='creativeID')
test=pd.merge(test,user,on='userID')
test=pd.merge(test,position,on='positionID')
test=pd.merge(test,app_categories,on='appID')
test['day']=test.clickTime.values/10000
test['hour']=test.clickTime.values/100%100
test['residence']=test.residence.values/100

In [3]:
def modelfit(alg, dtrain, dtest, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        xgtest = xgb.DMatrix(dtest[predictors].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='logloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['label'],eval_metric='logloss')
    #     Predict on testing data:
    dtest['predprob'] = alg.predict_proba(dtest[predictors])[:,1]

In [ ]:
target='label'
day='day'
id='instanceID'

In [ ]:
predictors = [x for x in train_all.columns if x not in [target,day]]
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=3,
        min_child_weight=5,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb1, train_all, test, predictors)

In [ ]:
import zipfile
df = pd.DataFrame({"instanceID": test["instanceID"].values, "proba":test["predprob"].values})
df.sort_values("instanceID", inplace=True)
df.to_csv("submission.csv", index=False)
with zipfile.ZipFile("submission.zip", "w") as fout:
    fout.write("submission.csv", compress_type=zipfile.ZIP_DEFLATED)

In [ ]:
1+2